# Dataset exploration

* [Annual mortality and causes by county, 1979-1988](https://www.cdc.gov/nchs/data_access/cmf.htm)
* [Compressed mortality info, 1968-2016](https://wonder.cdc.gov/controller/datarequest/D140)

[Rainfall and turnout](https://faculty.ucmerced.edu/thansford/Articles/The%20Republicans%20Should%20Pray%20for%20Rain%20-%20Weather,%20Turnour,%20and%20Voting%20in%20U.S.%20Presidential%20Elections.pdf)

[Higher temperatures increase suicide rates in the
United States and Mexico](https://web.stanford.edu/~mburke/papers/BurkeEtAl_NCC_2018.pdf)

As temperatures go up, suicide rates go up.
Heat waves, countries or regions that go through heat waves. During those times there were significant differences in suicide rates.

Is there monthly data?

[WHO mortality data](https://www.who.int/healthinfo/statistics/mortality_rawdata/en/)

[CDC Multiple Mortality Cause files](https://www.cdc.gov/nchs/data_access/vitalstatsonline.htm#Mortality_Multiple)

[Suicides and gun ownership](https://mason.gmu.edu/~atabarro/BriggsTabarrokFirearmsSuicide.pdf)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
sns.set_context("talk")
plt.style.use('ggplot')

## Read cleaned dataset from file

In [ ]:
def parser(x):
	return pd.datetime.strptime(x, '%Y/%m')

suicides = pd.read_csv('data/suicides_heat.csv', parse_dates=['Month Code'], index_col=0, date_parser=parser)
suicides

# Exploratory Data Analysis

In [ ]:
sns.barplot(x='State', y='suicide_rate', data=suicides)

In [ ]:
sns.scatterplot(x='avg_max_t', y='suicide_rate',  data=suicides.query('4 < Month < 10'), hue='Month', legend='full')

In [ ]:
sns.scatterplot(x='avg_max_heat_index', 
                y='suicide_rate', data=suicides.query('4 < Month < 10'), hue='Month', legend='full')

In [ ]:
sns.lmplot(x='avg_max_heat_index', 
                y='suicide_rate', data=suicides.query('4 < Month < 10'), legend='full')

In [ ]:
sns.scatterplot(x='heat_index_diff', y='suicide_rate', data=suicides.query('4 < Month < 10'), hue='Month')

What if we compare when heat_index diff > 5 to when heat_index diff < 5

In [ ]:
sns.lmplot(x='heat_index_diff', y='suicide_rate', data=suicides.query('4 < Month < 10'))

In [ ]:
sns.scatterplot(x='max_t_diff', y='suicide_rate', data=suicides.query('4 < Month < 10'), hue='Month')

In [ ]:
sns.scatterplot(x='min_t_diff', y='suicide_rate', data=suicides.query('4 < Month < 10'), hue='Month')

There may be a time effect over the years!

In [ ]:
sns.scatterplot(x='Month Code', y='suicide_rate', data=suicides[suicides.State == 'California'],hue='Month', legend='full')

There is a strong annual trend.

In [ ]:
plt.plot(suicides.groupby('Year').sum().Deaths)
plt.title('Suicides per Year in the US',size='x-large')
plt.xlabel('Year')
plt.ylabel('Suicides')

Chi-squared tells us whether two categorical variables are independent


understand occurences with one object and many factors
chi-squared test
```
                 99   | 00   | 01    | 02 | 03 ... 17
AL suicides      sum    sum    sum    sum   sum ...
CA 
CO
...
```

In [ ]:
in summer months: categorical variable low heat,  high heat 

## Hypothesis 1 - winter gets more suicides than summer

In [ ]:
summer = suicides.query('5 < Month < 9')
winter = suicides.query('Month < 4 or Month > 10')
spring = suicides.query('3 < Month < 6')
longsummer = suicides.query('3 < Month < 9')
fall = suicides.query('8 < Month < 11')

In [ ]:
sns.distplot(winter.suicide_rate, label='November-March')
sns.distplot(longsummer.suicide_rate, label='April-August')
# plt.hist(fall.suicide_rate, bins='auto', alpha=.5, label='September-October')
plt.legend()

In [ ]:
stats.ttest_ind(winter.suicide_rate, longsummer.suicide_rate, equal_var=False)

In [ ]:
# Cohen's d
def cohen_d(x,y):
    nx = len(x)
    ny = len(y)
    dof = nx + ny - 2
    return np.abs((np.mean(x) - np.mean(y)) / np.sqrt(((nx-1)*np.std(x, ddof=1) ** 2 + (ny-1)*np.std(y, ddof=1) ** 2) / dof))

cohen_d(winter.suicide_rate, longsummer.suicide_rate)

## Let's cancel out year effect

In [ ]:
vermont = suicides.query('State == "New York" & Month == 2').copy()
vermont

In [ ]:
vermont.query('Month == 12')

In [ ]:
# mean out the years
# observations are states
# group by states mean of rate by month
suicides_by_month = suicides.groupby(['Month',
                  'State']).agg('mean').reset_index() \
                .drop(columns=['Year','Year Code','min_t_diff', 'max_t_diff', 'heat_index_diff'])
suicides_by_month

In [ ]:
suicides_by_month['is_winter'] = np.where((suicides_by_month.Month < 4) | (suicides_by_month.Month > 10), 1, 0)

In [ ]:
X = suicides_by_month.dropna()[['Month','is_winter', 'avg_max_t', 'avg_min_t', 'avg_max_heat_index']].copy()
X = sm.add_constant(X)
Y = suicides_by_month.dropna()['suicide_rate']
model = sm.OLS(Y,X)
results = model.fit()
results.summary()

Paired t-test. Compare the sum of the months against the mean of the months

In [ ]:
summer_by_month = suicides_by_month.query('3 < Month < 9')
winter_by_month = suicides_by_month.query('Month < 4 or Month > 10')
sns.distplot(summer_by_month.suicide_rate, label='April-August')
sns.distplot(winter_by_month.suicide_rate, label='November-March')
plt.legend()
plt.rcParams["figure.figsize"] = (10,6)

plt.xlabel('suicide rate', size='x-large')
plt.title('Suicides Are Higher in Summer Than Winter',size=24)
sns.set_context("talk")
plt.style.use('ggplot')

In [ ]:
summer_by_month.groupby('State').mean()['suicide_rate']

In [ ]:
stats.ttest_rel(summer_by_month.groupby('State').mean()['suicide_rate'], 
                winter_by_month.groupby('State').mean()['suicide_rate'])

In [ ]:
cohen_d(summer_by_month.groupby('State').mean()['suicide_rate'], 
        winter_by_month.groupby('State').mean()['suicide_rate'])

In [ ]:
stats.ttest_rel(summer_by_month.groupby('State').sum()['suicide_rate'], 
                winter_by_month.groupby('State').sum()['suicide_rate'])

In [ ]:
cohen_d(summer_by_month.groupby('State').sum()['suicide_rate'], 
        winter_by_month.groupby('State').sum()['suicide_rate'])

Nope, in fact spring & summer are worse than winter!

## Looking at all our data

In [ ]:
sns.distplot(suicides.query('3 < Month < 9').avg_max_heat_index.dropna())

In [ ]:
sns.distplot(suicides[suicides.Year == 2009].query('3 < Month < 9').avg_max_heat_index.dropna())

In [ ]:
sns.distplot(suicides.query('3 < Month < 9').heat_index_diff.dropna())

## Extreme heat in summer

In [ ]:
q_75 = np.quantile(suicides.query('3 < Month < 9').avg_max_heat_index.dropna(), .75)
q_75

In [ ]:
q_05 = np.quantile(suicides.query('3 < Month < 9').avg_max_heat_index.dropna(), .05)
q_05

In [ ]:
q_95 = np.quantile(suicides.query('3 < Month < 9').avg_max_heat_index.dropna(), .95)
q_95

In [ ]:
low_heat = longsummer.query(f'avg_max_heat_index < {q_05}')
high_heat = longsummer.query(f'avg_max_heat_index > {q_95}')
stats.ttest_ind(low_heat.suicide_rate, high_heat.suicide_rate, equal_var=False)

## Unusually extreme heat has significant effect

In [ ]:
longsummer[['State','Year','Month','suicide_rate','heat_index_diff']].dropna()

In [ ]:
q_diff_05 = np.quantile(suicides.query('3 < Month < 9').heat_index_diff.dropna(), .05)
q_diff_05

In [ ]:
q_diff_95 = np.quantile(suicides.query('3 < Month < 9').heat_index_diff.dropna(), .95)
q_diff_95

In [ ]:
low_heat_diff = longsummer.query(f'heat_index_diff < {q_diff_05}')
high_heat_diff = longsummer.query(f'heat_index_diff > {q_diff_95}')
stats.ttest_rel(low_heat_diff.suicide_rate, high_heat_diff.suicide_rate)

In [ ]:
low_heat_diff = longsummer.query(f'heat_index_diff < {q_diff_05} & Year == 2009').dropna()
high_heat_diff = longsummer.query(f'heat_index_diff > {q_diff_95} & Year == 2009').dropna()
stats.ttest_ind(low_heat_diff.suicide_rate, high_heat_diff.suicide_rate, equal_var=False)

In [ ]:
longsummer.query('Year == 2009')

In [ ]:
longsummer.query(f'heat_index_diff < {q_diff_05}').suicide_rate.hist(alpha=.5, bins='auto', 
                                                                     label='5th percentile heat-index difference')
longsummer.query(f'heat_index_diff > {q_diff_95}').suicide_rate.hist(alpha=.5, bins='auto', 
                                                                     label='95th percentile heat-index difference')
plt.legend()
plt.style.use('seaborn-ticks')

In [ ]:
cohen_d(longsummer.query(f'heat_index_diff < {q_diff_05}').suicide_rate, 
        longsummer.query(f'heat_index_diff > {q_diff_95}').suicide_rate)

In [ ]:
def top_v_bottom(top, bottom, data, var, target, var_name="unusual heat"):
    data = data.dropna()
    q_top = np.quantile(data[var], top)
    q_bottom = np.quantile(data[var], bottom)
    top_target = data[data[var] >= q_top][target]
    bottom_target = data[data[var] <= q_bottom][target]
    sns.distplot(top_target, label=f'{top*100:.0f}th percentile\n{var_name}')
    sns.distplot(bottom_target, label=f'{bottom*100:.0f}th percentile\n{var_name}')
    plt.xlabel(target.replace("_"," "),size='xx-large')
    plt.legend()
    print(stats.ttest_ind(top_target, bottom_target, equal_var=False))
    print("Cohen's D score: ", cohen_d(bottom_target, top_target))
    
def diff_95_v_05(data, var, target):
    top_v_bottom(.95, .05, data, var, target)
    
diff_95_v_05(longsummer, 'heat_index_diff', 'suicide_rate')
plt.title("Suicide Rate Higher ")

In [ ]:
sns.distplot(longsummer.suicide_rate)

In [ ]:
top_v_bottom(.9, .1, suicides_by_month, 'avg_max_heat_index', 'suicide_rate')

In [ ]:
top_v_bottom(.95, .05, longsummer, 'avg_max_heat_index', 'suicide_rate')

In [ ]:
diff_95_v_05(longsummer, 'heat_index_diff', 'suicide_rate')

In [ ]:
top_v_bottom(.9, .1, longsummer, 'avg_max_heat_index', 'suicide_rate',var_name='heat index')
plt.title("Suicide Rate During\nWarm v. Cool Summer Months", size=24)

* hypothesis is that suicide rate during hot > suicide rate during cold.
* null is that it is not greater.
* 1-tailed test

scipy always gives the test statistic as signed. This means that given p and t values from a two-tailed test, you would reject the null hypothesis of a greater-than test when p/2 < alpha and t > 0, and of a less-than test when p/2 < alpha and t < 0.

In [ ]:
stats.ttest_ind(x, y), this makes a Hypothesis Test on the value of x.mean()-y.mean(), which means that in order to get positive values throughout the calculation (which simplifies all considerations) we have to call

stats.ttest_ind(longsummer[long,A)

In [ ]:
top_v_bottom(.9, .1, longsummer, 'heat_index_diff', 'suicide_rate')
plt.title("Suicide Rate During\nUnusually Warm v. Cool Summer Months", size=24)

In [ ]:
southeast = ("Alabama", "Florida", "Georgia", "Kentucky", "Mississippi", "North Carolina", "South Carolina", "Tennessee")
top_v_bottom(.9, .1, longsummer[longsummer.State.isin(southeast)], 'heat_index_diff', 'suicide_rate')

## Year is such a big factor. We can't ignore that

In [ ]:
top_v_bottom(.85, .15, longsummer[longsummer.State.isin(southeast)], 'Year', 'suicide_rate')

In [ ]:
sns.scatterplot(x='Month Code', y='suicide_rate',data=longsummer[longsummer.State.isin(southeast)])

In [ ]:
import statsmodels.api as sm

In [ ]:
X = longsummer.dropna()[['Year','Month','heat_index_diff', 'min_t_diff', 'max_t_diff']].copy()
X = sm.add_constant(X)
Y = longsummer.dropna()['suicide_rate']
model = sm.OLS(Y,X)
results = model.fit()
results.summary()

In [ ]:
results.tvalues

In [ ]:
top_v_bottom(.85, .15, longsummer[~longsummer.State.isin(southeast)], 'heat_index_diff', 'suicide_rate')

In [ ]:
northeast = ("Connecticut", 'Maine', "Massachusetts", "New Hampshire", "Rhode Island", "Vermont", "New Jersey", "New York", "Pennsylvania")
top_v_bottom(.9, .1, longsummer[longsummer.State.isin(northeast)], 'heat_index_diff', 'suicide_rate')
plt.title('Northeast',size='xx-large')

In [ ]:
top_v_bottom(.85, .15, longsummer[longsummer.State.isin(northeast)], 'avg_max_t', 'suicide_rate')

In [ ]:
longsummer[longsummer.State.isin(southeast)]

In [ ]:
suicides.query('3 < Month < 9').groupby('Year').avg_max_heat_index.hist(alpha=.25, bins='auto')

In [ ]:
suicides.query('3 < Month < 9').groupby('Year').heat_index_diff.hist(alpha=.25, bins='auto')

In [ ]:
sns.distplot(suicides.query('3 < Month < 9').heat_index_diff.dropna())

In [ ]:
np.quantile(suicides.query('3 < Month < 9').heat_index_diff.dropna(),.25)

In [ ]:
## Suicides by states vs years
suicides.groupby(['State', 'Year']).sum()['Deaths']

In [ ]:
suicides_by_state_and_year = pd.pivot_table(suicides, index='State', values='suicide_rate', columns='Year', aggfunc=np.mean).dropna()
suicides_by_state_and_year

In [ ]:
stats.chisquare(suicides_by_state_and_year)

In [ ]:
stats.chisquare(suicides_by_state_and_year.query('State in ("Alabama", "Mississippi")'))

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/2011_us_ag_exports.csv')

In [ ]:
suicides = suicides.merge(df[['state','code']], left_on='State', right_on='state').drop(columns='state')

In [ ]:
import plotly.graph_objects as go

In [ ]:
fig = go.Figure(data=go.Choropleth(
    locations=suicides.groupby('code').mean().index, # Spatial coordinates
    z = suicides.groupby('code').mean().suicide_rate, # Data to be color-coded
    locationmode = 'USA-states', # set of locations match entries in `locations`
    colorscale = 'Reds',
#    colorbar_title = "Millions USD",
))
fig.update_layout(
#    title_text = 'US Suicide Rates',
    geo_scope='usa', # limit map scope to USA
)
fig.show(renderer="png", width=1000, height=800)

In [ ]:
fig = go.Figure(data=go.Choropleth(
    locations=suicides.groupby('code').mean().index, # Spatial coordinates
    z = suicides.query('3 < Month < 9').groupby('code').mean().suicide_rate - suicides.query('Month < 4 or Month > 10').groupby('code').mean().suicide_rate, # Data to be color-coded
    locationmode = 'USA-states', # set of locations match entries in `locations`
    colorscale = 'Reds',
#    colorbar_title = "Millions USD",
))
fig.update_layout(
#    title_text = 'US Suicide Rates',
    geo_scope='usa', # limit map scope to USA
)
fig.show(renderer="png", width=1000, height=800)

In [ ]:
fig = go.Figure(data=go.Choropleth(
    locations=suicides.groupby('code').mean().index, # Spatial coordinates
    z = (suicides.query('Year == 2011').groupby('code').mean().suicide_rate 
        - suicides.query('Year == 1999').groupby('code').mean().suicide_rate) / suicides.query('Year == 1999').groupby('code').mean().suicide_rate, # Data to be color-coded
    locationmode = 'USA-states', # set of locations match entries in `locations`
    colorscale='RdBu',
    reversescale=True,
    zmid=0
#    colorbar_title = "Millions USD",
))
fig.update_layout(
    title_text = "Suicide Rate Change from 1999 to 2011",
    geo_scope='usa', # limit map scope to USA
)
fig.show(renderer="png", width=1000, height=800)

In [ ]:
fig = go.Figure(data=go.Choropleth(
    locations=suicides.groupby('code').mean().index, # Spatial coordinates
    z = suicides.groupby('code').mean().avg_max_heat_index, # Data to be color-coded
    locationmode = 'USA-states', # set of locations match entries in `locations`
    colorscale = 'Reds',
#    colorbar_title = "Millions USD",
))
fig.update_layout(
#    title_text = '2011 US Agriculture Exports by State',
    geo_scope='usa', # limit map scope to USA
)
fig.show(renderer="png", width=1000, height=800)

In [ ]:
fig = go.Figure(data=go.Choropleth(
    locations=suicides.groupby('code').mean().index, # Spatial coordinates
    z = suicides.groupby('code').mean().avg_max_t, # Data to be color-coded
    locationmode = 'USA-states', # set of locations match entries in `locations`
    colorscale = 'Reds',
#    colorbar_title = "Millions USD",
))
fig.update_layout(
#    title_text = '2011 US Agriculture Exports by State',
    geo_scope='usa', # limit map scope to USA
)
fig.show()

In [ ]:
fig = go.Figure(data=go.Choropleth(
    locations=suicides.query('3 < Month < 9').groupby('code').mean().index, # Spatial coordinates
    z = suicides.query('3 < Month < 9').groupby('code').std().avg_max_heat_index, # Data to be color-coded
    locationmode = 'USA-states', # set of locations match entries in `locations`
    colorscale = 'Reds',
#    colorbar_title = "Millions USD",
))
fig.update_layout(
#    title_text = '2011 US Agriculture Exports by State',
    geo_scope='usa', # limit map scope to USA
)
fig.show()

In [ ]:
fig = go.Figure(data=go.Choropleth(
    locations=suicides.query('3 < Month < 9').groupby('code').mean().index, # Spatial coordinates
    z = suicides.query('3 < Month < 9').groupby('code').std().avg_max_heat_index, # Data to be color-coded
    locationmode = 'USA-states', # set of locations match entries in `locations`
    colorscale = 'Reds',
#    colorbar_title = "Millions USD",
))
fig.update_layout(
#    title_text = '2011 US Agriculture Exports by State',
    geo_scope='usa', # limit map scope to USA
)
fig.show()

In [ ]:
suicides.groupby('code').mean().index

In [ ]:
q_diff_90 = np.quantile(suicides.query('3 < Month < 9').dropna().heat_index_diff, .9)
q_diff_90

In [ ]:
suicides.query(f'3 < Month < 9 & heat_index_diff >= {q_diff_90}').groupby('code').count()

In [ ]:
fig = go.Figure(data=go.Choropleth(
    locations=suicides.query(f'3 < Month < 9 & heat_index_diff >= {q_diff_90}').groupby('code').count().index, # Spatial coordinates
    z = suicides.query(f'3 < Month < 9 & heat_index_diff >= {q_diff_90}').groupby('code').count().heat_index_diff, # Data to be color-coded
    locationmode = 'USA-states', # set of locations match entries in `locations`
    colorscale = 'Reds',
#    colorbar_title = "Millions USD",
))
fig.update_layout(
#    title_text = '2011 US Agriculture Exports by State',
    geo_scope='usa', # limit map scope to USA
)
fig.show(renderer="png", width=1000, height=800)

In [ ]:
suicides.query(f'3 < Month < 9 & heat_index_diff >= {q_diff_90}').groupby('code').count().index

In [ ]:
plt.plot(suicides.query(f'3 < Month < 9 & heat_index_diff >= {q_diff_90}').groupby('Year').count().State)

In [ ]:
suicides.query(f'3 < Month < 9 & heat_index_diff >= {q_diff_90}')

## Detrend data experiment

In [ ]:
plt.plot(suicides.groupby('Month Code').sum().Deaths)

In [ ]:
from sklearn.linear_model import LinearRegression
# fit linear model
series = suicides.query('State == "Missouri"').copy()
X = [i for i in range(0, len(series))]
X = np.reshape(X, (len(X), 1))
y = series.suicide_rate.reset_index(drop=True)
model = LinearRegression()
model.fit(X, y)
# calculate trend
trend = model.predict(X)
# plot trend
plt.plot(y)
plt.plot(trend)
plt.show()


In [ ]:
print(f"best fit: y = {model.coef_[0]:.04f} x + {model.intercept_:.04f}")

In [ ]:
# detrend
detrended = [y[i]-trend[i] + y.mean() for i in range(0, len(series))] 
# plot detrended
plt.plot(detrended)
plt.show()

In [ ]:
series['detrended_suicide_rate'] = detrended

In [ ]:
series

In [ ]:
top_v_bottom(.90, .1, series.query('3 < Month < 9'), 'heat_index_diff', 'detrended_suicide_rate')
plt.title("Missouri detrended",size='xx-large')

In [ ]:
plt.plot(series.heat_index_diff)